In [ ]:

import numpy as np
import json
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [ ]:
with open("intents.json" ,'r') as f:
  intents = json.load(f)


In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies ar

In [ ]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))

ignore_words = ['?','!',".",',']
print(all_words)

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']


In [ ]:
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words)) # sorted and removing duplicates
tags = sorted(set(tags))
print(tags)
print((all_words))
print(xy)

['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
['a', 'age', 'am', 'anyon', 'are', 'buy', 'call', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodbi', 'guy', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'later', 'leav', 'like', 'menu', 'name', 'of', 'old', 'on', 'open', 'oper', 'reccommend', 'see', 'should', 'someth', 'the', 'there', 'tim', 'to', 'up', 'what', 'when', 'you', 'your']
[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'tim'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I

In [ ]:
X_train = []
y_train = []
for (pattern_sentence,tag) in xy:
  bag = bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)
  
  label = tags.index(tag)
  y_train.append(label) #Cross entrtopy loss

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
     return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    return self.n_samples
  
#Hyper parameters
batch_size = 8
num_epochs = 1000
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

dataset = ChatDataset()

train_loader = DataLoader(dataset = dataset,batch_size = batch_size, shuffle = True, num_workers = 2)

47 6


In [ ]:
#Model 

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


Epoch [100/1000], Loss: 0.8934
Epoch [200/1000], Loss: 0.2431
Epoch [300/1000], Loss: 0.0845
Epoch [400/1000], Loss: 0.0201
Epoch [500/1000], Loss: 0.0445
Epoch [600/1000], Loss: 0.0018
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0056
Epoch [900/1000], Loss: 0.0010
Epoch [1000/1000], Loss: 0.0005
final loss: 0.0005
training complete. file saved to data.pth


In [ ]:
#Chat
import random


data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model.load_state_dict(model_state)
model.eval()
bot_name = "Sam"

print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hello
Sam: Hi there, how can I help?
You: you are smart
Sam: Hey :-)
You: hey
Sam: Hello, thanks for visiting
You: how is it goin
Sam: Hello, thanks for visiting
You: how are you
Sam: Hey :-)
You: whats up
Sam: Have a nice day
You: you re bad
Sam: Bye! Come back again soon.
You: You are bad
Sam: Hi there, what can I do for you?
You: You can do a favour
Sam: I do not understand...
You: How Are You
Sam: Hi there, how can I help?
You: What do you sell
Sam: We sell coffee and tea
You: coffee please
Sam: Bye! Come back again soon.
You: Do you sell coffee ?
Sam: We sell coffee and tea
You: I want pasta
Sam: Have a nice day
You: I just have credit card
Sam: We accept most major credit cards, and Paypal
You: ok bye
Sam: Bye! Come back again soon.
You: quit
